In [1]:
from time import sleep
from pettingzoo.atari import mario_bros_v3

In [2]:
env = mario_bros_v3.env()
env.reset()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/gym/utils/seeding.py:155: DeprecationWarning: WARN: Function `create_seed(a, max_bytes)` is marked as deprecated and will be removed in the future. 
  deprecation(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/gym/utils/seeding.py:175: DeprecationWarning: WARN: Function `_bigint_from_bytes(bytes)` is marked as deprecated and will be removed in the future. 
  deprecation(


In [8]:
agent = 'first_0'
height, width, channels = env.observation_space(agent).shape
actions = env.action_space(agent).n
print(height,width,channels,actions)

210 160 3 18


In [18]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

In [33]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height,width,channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation = 'relu'))
    model.add(Dense(256, activation = 'relu'))
    model.add(Dense(actions, activation = 'linear'))
    return model

del model
model = build_model(height,width,channels,actions)


In [34]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 3, 51, 39, 32)     6176      
                                                                 
 conv2d_7 (Conv2D)           (None, 3, 24, 18, 64)     32832     
                                                                 
 conv2d_8 (Conv2D)           (None, 3, 22, 16, 64)     36928     
                                                                 
 flatten_2 (Flatten)         (None, 67584)             0         
                                                                 
 dense_11 (Dense)            (None, 512)               34603520  
                                                                 
 dense_12 (Dense)            (None, 256)               131328    
                                                                 
 dense_13 (Dense)            (None, 18)               

In [35]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy


def build_agent(model,actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(),attr='eps',value_max=1,value_min=0.1,value_test=0.2,nb_steps=10000)
    memory = SequentialMemory(limit=1000,window_length=3)
    dqn = DQNAgent(model = model, memory = memory, policy = policy,
                    enable_dueling_network=True, dueling_type='avg',
                    nb_actions = actions, nb_steps_warmup=10000
                    )
    return dqn

In [36]:
dqn = build_agent(model,actions)
dqn.compile(Adam(lr=1e-4))
dqn.fit(env,nb_steps=10000,visualize=False,verbose=2)

Training for 10000 steps ...


AssertionError: 